## Q1

In [ ]:
array = [1, 2, 1, 1, 2, 2, 1, 1]
s = 4

from pyspark import SparkContext
RDD = sc.parallelize(array, 2).cache()

def sequential(nums):
    # FILL IN YOUR CODE HERE
    global s 
    
    counter = 0 
    current_window = []
    for num in nums:
        
        current_window.append(num)
        summation = sum(current_window)
     
        if summation == s:
            counter += 1
            current_window.pop(0)
            continue
            
        elif summation < s:
            continue
            
        else: # >s 
            while sum(current_window) >= s:
                current_window.pop(0)
                if sum(current_window) == s:
                    counter += 1
                    break
    
    while sum(current_window) >= s:
        current_window.pop(0)
        if sum(current_window) == s:
            counter += 1
            break
    return counter
            
    

def divide(it):
    # FILL IN YOUR CODE HERE
    
    it = list(it) #
    counter = sequential(it)
    
    L,R = [], []
    indicator = True
    for i in it:
        if sum(L) + i < s and indicator:
            L.append(i) 
        else:
            indicator = False
        R.append(i)
        # print(L)
        # print(R)
        
    while sum(R) >= s:
        R.pop(0)
        
    yield counter,L,R

def conquer(X):
    # FILL IN YOUR CODE HERE
    n = len(X)
    counter = 0
    for i in range(n):
        if i == 0:
            counter += X[i][0]
            continue
        else:
            counter += X[i][0]
            tmp = X[i-1][2]
            tmp.extend(X[i][1])
            counter += sequential(tmp)
    return counter

X = RDD.mapPartitions(divide).collect()
result = conquer(X)

# END OF YOUR CODE

### Q2

In [ ]:
from graphframes import *
from pyspark.sql.functions import *

# copy v and e from the previous example
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "b", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])

# FILL IN YOUR CODE HERE

# Create a GraphFrame
g = GraphFrame(v, e)

src_follow_count = g.edges.filter("relationship = 'follow' ")\
                    .groupBy('src')\
                    .count()\
                    .withColumnRenamed('count', 'count_follow')\
                    .withColumnRenamed('src', 'id')
#a float value indicates the average following
follow_avg = src_follow_count.agg(avg(col("count_follow")))\
                             .collect()[0]\
                              .asDict()['avg(count_follow)']

dst_follow_count = g.edges.filter("relationship = 'follow' ")\
                    .groupBy('dst')\
                    .count()\
                    .withColumnRenamed('dst', 'id')\
                    .withColumnRenamed('count', 'count_followed')

candidate_id = src_follow_count.join(dst_follow_count,'id')\
                .filter("'count_follow' < 'count_followed' ")\
                .filter(col('count_follow') > lit(follow_avg))\
                .select('id')
# REPORT THE RESULT HERE

result = v.join(candidate_id,'id').select('name')

result.show()

# END OF YOUR COD

### Q3

In [ ]:
from graphframes import *
from pyspark.sql.functions import *

# copy v and e from the previous example
v = spark.createDataFrame([
 ("a", "Alice", 34),
 ("b", "Bob", 36),
 ("c", "Charlie", 37),
 ("d", "David", 29),
 ("e", "Esther", 32),
 ("f", "Fanny", 38),
 ("g", "Gabby", 60)
], ["id", "name", "age"])
e = spark.createDataFrame([
 ("a", "b", "follow"),
 ("a", "c", "friend"),
 ("a", "g", "friend"),
 ("b", "c", "friend"),
 ("c", "a", "friend"),
 ("c", "b", "friend"),
 ("c", "d", "follow"),
 ("c", "g", "friend"),
 ("d", "a", "follow"),
 ("d", "b", "follow"),
 ("d", "g", "friend"),
 ("e", "a", "follow"),
 ("e", "d", "follow"),
 ("f", "b", "follow"),
 ("f", "c", "follow"),
 ("f", "d", "follow"),
 ("g", "a", "friend"),
 ("g", "c", "friend"),
 ("g", "d", "friend")
], ["src", "dst", "relationship"])

from graphframes import *
g = GraphFrame(v, e)

# FILL IN YOUR CODE HERE
from pyspark.sql.functions import udf

# REPORT THE RESULT HERE
triangle = g.find("(a)-[e1]->(b); (b)-[e2]->(c); (c)-[e3]->(a)")\
 .filter(" e1.relationship = 'friend' and e2.relationship = 'friend' and e3.relationship = 'friend'")\
 .filter("a.id < b.id and a.id < c.id and b.id < c.id")\

get_name = udf(lambda s: s[1])
result = triangle.select(get_name(triangle['a']).alias('name'),
               get_name(triangle['b']).alias('name'),
               get_name(triangle['c']).alias('name'))

# result.show()

# END OF YOUR CODE

In [1]:
array = [1, 2, 1, 1, 2, 2, 1, 1]
s = 4

from pyspark import SparkContext
RDD = sc.parallelize(array, 2).cache()

def sequential(nums):
    # FILL IN YOUR CODE HERE
    global s 
    
    counter = 0 
    current_window = []
    for num in nums:
        
        current_window.append(num)
        summation = sum(current_window)
     
        if summation == s:
            counter += 1
            current_window.pop(0)
            continue
            
        elif summation < s:
            continue
            
        else: # >s 
            while sum(current_window) >= s:
                current_window.pop(0)
                if sum(current_window) == s:
                    counter += 1
                    break
    
    while sum(current_window) >= s:
        current_window.pop(0)
        if sum(current_window) == s:
            counter += 1
            break
    return counter
            
    

def divide(it):
    # FILL IN YOUR CODE HERE
    
    it = list(it) #
    counter = sequential(it)
    
    L,R = [], []
    indicator = True
    for i in it:
        if sum(L) + i < s and indicator:
            L.append(i) 
        else:
            indicator = False
        R.append(i)
        # print(L)
        # print(R)
        
    while sum(R) >= s:
        R.pop(0)
        
    yield counter,L,R

def conquer(X):
    # FILL IN YOUR CODE HERE
    n = len(X)
    counter = 0
    for i in range(n):
        if i == 0:
            counter += X[i][0]
            continue
        else:
            counter += X[i][0]
            tmp = X[i-1][2]
            tmp.extend(X[i][1])
            counter += sequential(tmp)
    return counter

X = RDD.mapPartitions(divide).collect()
result = conquer(X)

# END OF YOUR COD